In [56]:
import pandas as pd
import matplotlib.pyplot as plt       # Visualizations
import seaborn as sns                 # Visualizations
import numpy as np
from sklearn.metrics import roc_auc_score
from collections import Counter

In [57]:
df= pd.read_csv("../用户点击广告行为预测竞赛数据/train.csv").set_index('ID')
label= pd.read_csv("../用户点击广告行为预测竞赛数据/train_label.csv").set_index('ID')
#X_test = pd.read_csv("../用户点击广告行为预测竞赛数据/test.csv")
#X['date'] = pd.to_datetime(X['date'])


In [58]:
df.head()

,date,A1,A2,A3,B1,B2,B3,C1,C2,C3,...,E20,E21,E22,E23,E24,E25,E26,E27,E28,E29
ID,,,,,,,,,,,,,,,,,,,,,
0,1900-07-01 23:30:00,-7834936860748470404,2998835353230292833,-7494990137288550401,-8639208079192601888,8626319289109649330,-5533056078568352733,-2379289153703677865,-6705654019294684257,-2852474418297023562,...,0,0.266158,-1.600320,19,14,0,10,8,10,22
2,1900-07-01 23:30:00,5078873087020104664,544983809622051205,5835254758531463848,-8639208079192601888,8626319289109649330,-5533056078568352733,2501940309998358162,-6705654019294684257,8768516826754530049,...,0,0.266158,-1.600320,17,14,0,10,6,10,20
3,1900-07-01 23:30:00,-6322717615964561015,5363506900520348449,-1424055534205623346,-8639208079192601888,8626319289109649330,-5533056078568352733,1444408531941919970,-6705654019294684257,-5090034199475073432,...,0,0.266158,0.619259,17,6,0,4,6,4,20
5,1900-07-01 23:30:00,4650039790794677128,2774817855788439768,9040931202870639290,-8639208079192601888,8626319289109649330,-5533056078568352733,3524367011807253962,-6705654019294684257,453723491886073494,...,1,0.266158,0.619259,17,6,0,4,2,4,20
7,1900-07-01 23:30:00,-6322717615964561015,5363506900520348449,-1424055534205623346,-8639208079192601888,8626319289109649330,-5533056078568352733,2472798206889362914,-6705654019294684257,7637484814727325562,...,0,0.266158,0.619259,17,6,0,4,6,4,20


## 空值、重复值、时间序列数据处理

In [45]:
df['date'] = pd.to_datetime(df['date'])
df['F2'] = df['date'].dt.day
df['F3'] = df['date'].dt.hour   
df = df.drop(['date'], axis=1)
df = df.join(label,on='ID')

df = df.drop_duplicates()
df.iloc[:,:-1] = df.iloc[:,:-1].fillna(df.interpolate())

## 数据缩放

In [46]:
from sklearn.preprocessing import StandardScaler
dfx = np.array(df.iloc[:,:-1])

std = StandardScaler()
dfx = std.fit_transform(dfx)
#dfx = (dfx-np.mean(dfx))/(np.max(dfx)-np.min(dfx))

for i in range(0,df.shape[1]-1):
    df.iloc[:,i] = dfx[:,i]

In [47]:
# # Create correlation matrix
# corr_matrix = df.corr().abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# # Find index of feature columns with correlation greater than 0.95
# to_drop = [column for column in upper.columns if any(upper[column] > 0.99)]
# print(to_drop)


In [48]:
## 高度相关数据删除
df = df.drop(['E19', 'E22', 'E26'],axis=1)

In [49]:
X = np.array(df.drop('label',axis=1))
Y = np.array(df['label'])

In [50]:
X.shape

(59781, 39)

In [51]:
one_hot_columns = [
    'D1','D2','E4',
    'E8','E11','E13',
    'E16','E18','E21',
    'E25'
]

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(categories='auto')
df = df.join(pd.DataFrame (enc.fit_transform(df[one_hot_columns]).toarray()))

## 负样本添加 

In [52]:
'''Smote 添加'''
from imblearn.over_sampling import SMOTE,ADASYN
X_smote,Y_smote = SMOTE(kind = 'borderline1').fit_sample(X,Y)
print(sorted(Counter(Y_smote).items()))

[(0, 49544), (1, 49544)]


In [53]:
# from imblearn.datasets import make_imbalance 
# ratio = {0:10237,1:10237}
# X_mb,Y_mb = make_imbalance (X,Y,ratio = ratio)
# print(sorted(Counter(Y_mb).items()))

## 独热

In [55]:
feature = list(df.columns)
for idx,feat in enumerate(feature):
    if idx == 0:
        newdf = pd.DataFrame({feat:X_smote[:,idx]})
    else :
        newdf = newdf.join(pd.DataFrame({feat:X_smote[:,idx]}))
newdf = newdf.join(pd.DataFrame({'label':Y_smote}))

IndexError: index 39 is out of bounds for axis 1 with size 39

In [ ]:
newdf.shape

In [25]:
newdf = newdf.drop('label',axis=1)

In [29]:
df.shape

(59781, 40)

In [28]:
newdf.shape

(99088, 1935)

## LR 训练

In [536]:
from sklearn.model_selection import train_test_split,GridSearchCV
X_train, X_test, Y_train, Y_test = \
    train_test_split(X_enc,Y_enc, test_size=0.3)

# from sklearn.model_selection import train_test_split
# X_train, X_test, Y_train, Y_test = \
#     train_test_split(X_smote,Y_smote, test_size=0.3)

In [540]:
from sklearn.linear_model import LogisticRegression
log = GridSearchCV(LogisticRegression(),{'C':[1,0.1,10],'dual':[True],'max_iter':[1000]},cv=5)
log.fit(X_train,Y_train)

/Users/prettybeach/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/prettybeach/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/prettybeach/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/prettybeach/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/prettybeach/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solv

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 0.1, 10], 'dual': [True], 'max_iter': [1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [547]:
roc_auc_score(Y_test,log.predict_proba(X_test)[:,1])

0.6832022529910808

## 保存数据 

In [555]:
newdf.to_csv("../用户点击广告行为预测竞赛数据/my_train.csv")

In [ ]:
##### 使用特征工程做第二部分  #######